In [277]:
import numpy as np
import random
from DeepESN import DeepESN
from utils import MSE, load_chest, select_indexes
import copy
class Struct(object): pass

from sklearn.model_selection import ParameterGrid

np.random.seed(7)

In [278]:
def train(Nr, Nl, reg, transient, rhos, lis, iss, con):
    configs = Struct()
 
    configs.rhos = rhos
    configs.lis = lis
    configs.iss = iss

    configs.IPconf = Struct()
    configs.IPconf.DeepIP = 0 # deactivate pre-train

    configs.reservoirConf = Struct()
    configs.reservoirConf.connectivity = con 

    configs.readout = Struct()
    configs.readout.trainMethod = 'SVD' # train with singular value decomposition (more accurate)
    configs.readout.regularizations = 10.0**np.array(range(-16,-1,1))

    deepESN = DeepESN(Nu, Nr, Nl, configs)
    states = deepESN.computeState(dataset.inputs, deepESN.IPconf.DeepIP)
        
    train_states = select_indexes(states, list(TR_indexes) + list(VL_indexes), transient)
    train_targets = select_indexes(dataset.targets, list(TR_indexes) + list(VL_indexes), transient)
    test_states = select_indexes(states, TS_indexes)
    test_targets = select_indexes(dataset.targets, TS_indexes)
    
    deepESN.trainReadout(train_states, train_targets, reg)

    return score(deepESN, train_states, test_states, test_targets)

In [279]:
def score(deepESN, train_states, test_states, test_targets):
    #compute error in standard way
    test_states_std = copy.deepcopy(test_states)
    test_outputs_std = deepESN.computeOutput(test_states_std)
    test_error_std = MSE(test_outputs_std, test_targets)

    #compute error in our way
    test_outputs = []
    state = [train_states[0][:,-1:]] #set last training state as initial state
    output = [np.array(deepESN.computeOutput(state))] #set last training output initial output
    for t in range(len(test_states[0][0])):
        state = deepESN.computeState(inputs=output, DeepIP=0, initialStates=state[0])
        output = deepESN.computeOutput(state)
        test_outputs.append(output)
        output = [np.array(output)]
#             if t % 3000 == 0:
#                 print(t)
#                 print(output)

    test_error = MSE(np.array(test_outputs), np.array(test_targets))
    return test_error, test_outputs, test_error_std, test_outputs_std

In [280]:
test = train(10, 2, 0.00, 100, 0.9, 1.0, 0.1, 1)

In [281]:
#construct parameter grid

#set parameter ranges for sweep
parameters = {
    "Nr": [10, 100, 200],          # number of recurrent units
    "Nl": [1, 2, 5, 10],                     # number of recurrent layers
    "reg": [0.0, 0.1, 1.0],                  # regularization of svd
    "transient": [1000],                 # washout
    "rhos": [0.2, 0.5, 1.0],       # set spectral radius for all recurrent layers
    "lis": [0.1, 0.2, 0.5],            # set leaky rate for all recurrent layers
    "iss": [0.1, 1.0],               # set input scale for all recurrent layers
    "con": [0.2, 0.5, 1.0],                 # connectivity of recurrent matrix
    ".sampling": [1,10,25]             # set step size for sampling, 1 is no sampling. be careful not to make dataset too small
}

data_size = 20000 # pick dataset length to run parameter sweep on

if 640640/max(parameters[".sampling"]) < data_size:
    print("WARNING: highest sampling rate means that resulting dataset is smaller than data_size. \n")

print("Training data length: " + str(int(data_size*0.5)) + "  Validating data length: " + str(int(data_size*0.5)))

search_space = ParameterGrid(parameters)
print("number of models to be swept over: " + str(len(search_space)))
for i in range(10):
    print(search_space[i])
    

Training data length: 10000  Validating data length: 10000
number of models to be swept over: 5832
{'transient': 1000, 'rhos': 0.2, 'reg': 0.0, 'lis': 0.1, 'iss': 0.1, 'con': 0.2, 'Nr': 10, 'Nl': 1, '.sampling': 1}
{'transient': 1000, 'rhos': 0.5, 'reg': 0.0, 'lis': 0.1, 'iss': 0.1, 'con': 0.2, 'Nr': 10, 'Nl': 1, '.sampling': 1}
{'transient': 1000, 'rhos': 1.0, 'reg': 0.0, 'lis': 0.1, 'iss': 0.1, 'con': 0.2, 'Nr': 10, 'Nl': 1, '.sampling': 1}
{'transient': 1000, 'rhos': 0.2, 'reg': 0.1, 'lis': 0.1, 'iss': 0.1, 'con': 0.2, 'Nr': 10, 'Nl': 1, '.sampling': 1}
{'transient': 1000, 'rhos': 0.5, 'reg': 0.1, 'lis': 0.1, 'iss': 0.1, 'con': 0.2, 'Nr': 10, 'Nl': 1, '.sampling': 1}
{'transient': 1000, 'rhos': 1.0, 'reg': 0.1, 'lis': 0.1, 'iss': 0.1, 'con': 0.2, 'Nr': 10, 'Nl': 1, '.sampling': 1}
{'transient': 1000, 'rhos': 0.2, 'reg': 1.0, 'lis': 0.1, 'iss': 0.1, 'con': 0.2, 'Nr': 10, 'Nl': 1, '.sampling': 1}
{'transient': 1000, 'rhos': 0.5, 'reg': 1.0, 'lis': 0.1, 'iss': 0.1, 'con': 0.2, 'Nr': 10

In [292]:
search_space_short = []
for i in range(25):
    index = round(random.random() * len(search_space))
    search_space_short.append(search_space[index])
    
search_space_short = sorted(search_space_short, key=lambda k: k['.sampling'])
search_space_short

[{'transient': 1000,
  'rhos': 1.0,
  'reg': 0.1,
  'lis': 0.1,
  'iss': 1.0,
  'con': 0.2,
  'Nr': 200,
  'Nl': 10,
  '.sampling': 1},
 {'transient': 1000,
  'rhos': 1.0,
  'reg': 0.0,
  'lis': 0.5,
  'iss': 1.0,
  'con': 1.0,
  'Nr': 200,
  'Nl': 5,
  '.sampling': 1},
 {'transient': 1000,
  'rhos': 0.2,
  'reg': 1.0,
  'lis': 0.2,
  'iss': 1.0,
  'con': 0.5,
  'Nr': 200,
  'Nl': 1,
  '.sampling': 10},
 {'transient': 1000,
  'rhos': 0.2,
  'reg': 0.0,
  'lis': 0.2,
  'iss': 0.1,
  'con': 0.5,
  'Nr': 100,
  'Nl': 1,
  '.sampling': 25}]

In [293]:
# run the parameter sweep
import time

start = time.time()
results = []
previous = None

for i, args in enumerate(search_space_short):

    if args[".sampling"] != previous:
        #train-test data split 50/50
        dataset, Nu, TR_indexes, VL_indexes, TS_indexes = load_chest('datasets', data_size, args[".sampling"])
        previous = args[".sampling"]
    results.append(train(Nr=args['Nr'], Nl=args['Nl'], reg=args['reg'], transient=args['transient'], rhos=args['rhos'], lis=args['lis'], iss=args['iss'], con=args['con']))

    print(i)

print("average time taken: " + str((time.time()-start)/len(search_space_short)))

input length =  19999
.7 =  13999
.8 =  15999
-1 =  19998
Done loading data
0
1
input length =  19999
.7 =  13999
.8 =  15999
-1 =  19998
Done loading data
2
input length =  19999
.7 =  13999
.8 =  15999
-1 =  19998
Done loading data
3
average time taken: 17.847159564495087


In [301]:
keys = []
for key in search_space_short[0].keys():
    keys.append(key)
keys.append("result[0]")
keys.append("result[1]")
keys.append("result[2]")
keys.append("result[3]")

header = keys

with open('results.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(header)

    for i in range(len(search_space_short)):
        vals = []
        for val in search_space_short[i].values():
            vals.append(val)
        for j in range(len(results[0])):
            vals.append(results[j])
        write.writerow(vals)
